In [2]:
use std::fmt;

#[derive(Debug, Clone, Copy, PartialEq, Eq, Hash)]
pub enum Suit {
    Man, // Manzu
    Pin, // Pinzu
    Sou, // Souzu
}

#[derive(Debug, Clone, Copy, PartialEq, Eq, Hash)]
pub enum Honor {
    East, South, West, North,
    White, Green, Red,
}

#[derive(Debug, Clone, Copy, PartialEq, Eq, Hash)]
pub enum Tile {
    Suited { suit: Suit, value: u8 }, // value will be 1..9
    Honor(Honor),
}

In [3]:
impl TryFrom<&str> for Tile {
    type Error = String;

    fn try_from(input: &str) -> Result<Self, Self::Error> {
        // Convert the string to a list of characters
        let chars: Vec<char> = input.chars().collect();
        
        // Basic validation: "1m" or "ew" are 2 characters
        if chars.len() != 2 {
            return Err(format!("Invalid tile format: {}", input));
        }

        let value_char = chars[0];
        let type_char = chars[1];

        // TASK: Use a 'match' statement here to handle type_char
        // 'm' -> Suit::Man
        // 'p' -> Suit::Pin
        // 's' -> Suit::Sou
        // 'z' -> This is usually how honors are represented (1z = East, etc.)
        
        match type_char {
            'm' | 'p' | 's' => {
                let suit = match type_char {
                    'm' => Suit::Man,
                    'p' => Suit::Pin,
                    _ => Suit::Sou,
                };
                
                // Convert value_char to a digit (1-9)
                let val = value_char.to_digit(10).ok_or("Not a digit")? as u8;
                
                if val < 1 || val > 9 {
                    return Err("Suited tiles must be 1-9".to_string());
                }
                
                Ok(Tile::Suited { suit, value: val })
            },
            'z' => {
                // Map 1-7 to the Honor enum variants
                // 1=East, 2=South, 3=West, 4=North, 5=White, 6=Green, 7=Red
                // 1. Create a variable for the specific Honor variant
                let val_digit = value_char.to_digit(10).ok_or("Not a digit")? as u8;
                // 2. Use a match on val_digit (1 through 7)
                let honor_variant = match val_digit {
                    1 => Honor::East,
                    2 => Honor::South,
                    3 => Honor::West,
                    4 => Honor::North,
                    5 => Honor::White,
                    6 => Honor::Green,
                    7 => Honor::Red,
                    // ... and so on ...
                    _ => return Err("Out of bounds for Honors".into()),
                };
            
                // 3. Wrap that variant into the Tile enum and return it successfully
                Ok(Tile::Honor(honor_variant))
            },
            _ => Err(format!("Unknown suit: {}", type_char)),
        }
    }
}

In [4]:
// Test Success
let t1 = Tile::try_from("1m");
let t2 = Tile::try_from("7z");
println!("Success 1: {:?}", t1);
println!("Success 2: {:?}", t2);

// Test Failure
let e1 = Tile::try_from("0m"); // Invalid number
let e2 = Tile::try_from("1x"); // Invalid suit
println!("Error 1: {:?}", e1);
println!("Error 2: {:?}", e2);

Success 1: Ok(Suited { suit: Man, value: 1 })
Success 2: Ok(Honor(Red))
Error 1: Err("Suited tiles must be 1-9")
Error 2: Err("Unknown suit: x")


In [5]:
use std::collections::HashMap;

fn parse_hand(input: Vec<&str>) -> Result<HashMap<Tile, u8>, String> {
    // 1. Hand Size Validation
    if input.len() != 14 {
        return Err(format!("Hand must have 14 tiles, found {}", input.len()));
    }

    let mut counts = HashMap::new();

    // 2. Aggregate Counts
    for tile_str in input {
        // The '?' here is powerful: if TryFrom fails, the whole function 
        // returns early with that Error!
        let tile = Tile::try_from(tile_str)?; 
        
        // Entry API: This is the idiomatic way to update a map.
        // It's safe, efficient, and avoids double-lookups.
        let count = counts.entry(tile).or_insert(0);
        *count += 1;
    }

    // 3. Legal Limit Validation (Max 4 of each tile)
    for (tile, &count) in counts.iter() {
        if count > 4 {
            return Err(format!("Illegal hand: tile {:?} found {} times", tile, count));
        }
    }

    Ok(counts)
}

// --- TEST IT ---
let valid_input = vec![
    "1m", "2m", "3m", "1p", "2p", "3p", "1s", "2s", "3s", "1z", "1z", "1z", "2z", "2z"
];

match parse_hand(valid_input) {
    Ok(counts) => println!("Success! Total unique tiles: {}", counts.len()),
    Err(e) => println!("Error: {}", e),
}

Success! Total unique tiles: 11


()

In [6]:
// The "Pretty Printer"
// This lets you use println!("{}", tile) instead of println!("{:?}", tile)
impl fmt::Display for Tile {
    fn fmt(&self, f: &mut fmt::Formatter) -> fmt::Result {
        match self {
            Tile::Suited { suit, value } => {
                let s = match suit {
                    Suit::Man => 'm',
                    Suit::Pin => 'p',
                    Suit::Sou => 's',
                };
                write!(f, "{}{}", value, s)
            }
            Tile::Honor(h) => {
                let v = match h {
                    Honor::East => 1,
                    Honor::South => 2,
                    Honor::West => 3,
                    Honor::North => 4,
                    Honor::White => 5,
                    Honor::Green => 6,
                    Honor::Red => 7,
                };
                write!(f, "{}z", v)
            }
        }
    }
}

// TEST IT
let t = Tile::try_from("5z").unwrap();

In [7]:
println!("Parsed tile: {}", t); // Should print "5z"

Parsed tile: 5z


In [9]:
let valid_input = vec![
    "1m", "2m", "3m", "1p", "2p", "3p", "1s", "2s", "3s", "1z", "1z", "1z", "2z", "2z"
];
let blog = parse_hand(valid_input);

In [10]:
blog

Ok({Honor(East): 3, Suited { suit: Pin, value: 2 }: 1, Suited { suit: Man, value: 2 }: 1, Honor(South): 2, Suited { suit: Man, value: 3 }: 1, Suited { suit: Pin, value: 1 }: 1, Suited { suit: Man, value: 1 }: 1, Suited { suit: Pin, value: 3 }: 1, Suited { suit: Sou, value: 1 }: 1, Suited { suit: Sou, value: 2 }: 1, Suited { suit: Sou, value: 3 }: 1})